In [30]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

import matplotlib.pyplot as plt
import seaborn as sns

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

**Please run this file on Kaggle.com , use 'https://www.kaggle.com/datasets/kaiju8/cerebro-kaiju' and add it in input folder**
**Or do anything equivalent to import all libraries and read 'data' and 'testx'**

In [31]:
data = pd.read_csv("../input/cerebro-kaiju/weather_train.csv",index_col=0)
data.head()

Firstly I read data(training data) from the already imported folder

In [32]:
#Dropping all the rows where snow depth is negative
data.drop(data[(data['Snow Depth [cm]'] <0)].index, inplace=True)
data.head()

Before moving further, all the data containing negative snow depth is dropped(This improved the score significantly)

**Heat Map**

In [33]:
corr = data.corr()
fig, ax = plt.subplots(figsize=(10, 10))

sns.heatmap(corr, cmap='RdBu', annot=True, fmt=".2f")
plt.xticks(range(len(corr.columns)), corr.columns);
plt.yticks(range(len(corr.columns)), corr.columns)


plt.show()  

**Pairplot**

In [34]:
#sns.pairplot(data)  
#plt.show()

Visualising data to get an insight

In [35]:
data.describe()

Looking for absurdities in data(none found)

Firstly we will drop emtpty cells, prior to which we check number of empty cells

In [36]:
data.isnull().sum() 

Displaying all the dupliacte data

In [37]:
data[data.duplicated(keep=False)]

> No duplicate rows or empty cells were found

In [38]:
print(list(data.columns))

In [39]:
data.rename(columns = {'Snow Depth [cm]':'snowdepth'}, inplace = True)

Renaming Snow depth column for easier access

**Scaling of this data for some reason was increasing the score, so I decided to not do it**

Extracting Independent and dependent Variable 

In [40]:
#DATE and  MST are data types which I think would be ineffective here.

features = ['Global CMP22 (vent/cor) [W/m^2]', 'Direct sNIP [W/m^2]','Azimuth Angle [degrees]', 'Tower Dry Bulb Temp [deg C]', 'Tower Wet Bulb Temp [deg C]', 'Tower Dew Point Temp [deg C]', 'Tower RH [%]', 'Peak Wind Speed @ 6ft [m/s]', 'Avg Wind Direction @ 6ft [deg from N]', 'Station Pressure [mBar]', 'Precipitation (Accumulated) [mm]', 'Moisture', 'Albedo (CMP11)', 'Cloud Type', 'Solar Zenith Angle']
x= data[features]
y= data.snowdepth
x.describe()

This model yielded best results among Linear Regressor, Polynomial Regressor, Random Forest Regressor
(The reason behind Random Forest and Decision Tree regressor is due to The 'Intro to ML' course on Kaggle)

I also tried to a apply a deep learning algorithm and SVM, but both failed at some stage and got too complicated so I gave up

If needed I can share the file for other models

In [41]:
from sklearn.tree import DecisionTreeRegressor

regressor_tree_decision = DecisionTreeRegressor(random_state=1)

# Fit model
regressor_tree_decision.fit(x, y)

In [42]:
testx = pd.read_csv("../input/cerebro-kaiju/weather_test.csv",index_col=0)
testx.tail()

t= testx[features]

In [43]:
prediction = regressor_tree_decision.predict(t.head(2406))
ID = np.arange(0,2406)

pred=pd.DataFrame({"Id": ID,"Snow Depth [cm]": prediction})

pred.to_csv('mycsvfile.csv',index=False)

Collect the output csv file from kaggle/working folder

The below data is based on the Score I got upon submission on the Kaggle Beginner Hypothesis Competition (you can find it under kaiju8 profile)

In [51]:
score = pd.DataFrame({'Model':['Mean','LR','PR','DTR(This file)','RFR',"DTR-'column'","'DTR-'row'"],
         'Score':[2.43205,1.91898,1.49951,0.66324,0.66664,0.76773,0.82781]})

score.plot.bar(x ='Model', y='Score', rot=0)

plt.show()


Mean was just submitting mean of all the snow depth in test data(after dropping negative values)<br>
LR is Linear Regression<br>
PR is Polynomial Regression(here it was just second degree)<br>
DTR is Decision Tree Regressor<br>
RFR is Random Forest Regressor<br>
DTR-'column' is DTR after dropping two column with angle as data<br>
DTR-'row' is DTR after dropping rows with somewhat absurd value in column 'Direct sNIP [W/m^2]'

The Reason I believe Linear Regression and Polynomial Regression shows low accuracy is because their limited flexibility compared to DTR and RFR which are much advanced in nature.<br>
I also think that Deep Learning method might have shown a better accuracy, but because I ran into too many errors in its formation I did not finish it